### An example of the data model


According to the information on datasets that mentioned:

We have 'one to optional many relationship' from climate to hotspot data. For modelling this relationship we have two options namely embedding and referencing with their own pros and cons. Generally if the data on the 'many' side have too many records for each record on the 'one' side of the relationship we should choose referencing model  in order to maintain the 16 Mb restriction on MongoDb document. However, during analysis I found out that there are as many as 400 hotspot records for one climate record. Now during  research, I also found that and even if I consider hotspot records to increase to a few thousand records per climate record, still we will be well within the 16 Mb limit.

Most climate records have zero fire records with a handful exceptions in the historic CSVs. Moreover, in the later part, when we deal with streams, we will have 'one to optional one' relationship from climate to fire records!

These reasons coupled with the fact that referencing documents make queries a bit more time consuming and StopFire Campaign is already unable to produce results on time! So, while designing the model we should give 'time' the utmost importance in order to prevent fires (or at least reduce damage). Therefore, I chose to implement Embedding Model for this application.

One more point to be highlighted is that there were some standalone queries on the hotspot data like 'Find the top 10 records with highest surface temperature'. Now if this information is embedded in the climate data, it will take a bit more time for querying as compared if the hotspot data is referenced in the the climate data, but this is a trade off that we had to make to make the overall application more efficient.

So, in each climate document, we will have a 'hotspot' key which will contain array of all the respective fire records if any.

An example of the data model is shown below:

<pre>
[
 {'_id': 'c63',
  'air_temperature_celcius': 20,
  'date': datetime.datetime(2017, 3, 5, 0, 0),
  'hotspots': [],
  'max_wind_speed': 15.9,
  'precipitation': '0.00I',
  'relative_humidity': 58.8,
  'station': '948701',
  'windspeed_knots': 11.5},
 {'_id': 'c64',
  'air_temperature_celcius': 20,
  'date': datetime.datetime(2017, 3, 6, 0, 0),
  'hotspots': [{'_id': 'h2666',
                'confidence': 87,
                'date': datetime.datetime(2017, 3, 6, 0, 0),
                'datetime': datetime.datetime(2017, 3, 6, 5, 6, 30),
                'latitude': -34.3795,
                'longitude': 141.6331,
                'surface_temperature_celcius': 62},
               {'_id': 'h2667',
                'confidence': 85,
                'date': datetime.datetime(2017, 3, 6, 0, 0),
                'datetime': datetime.datetime(2017, 3, 6, 5, 6, 20),
                'latitude': -34.3735,
                'longitude': 141.6604,
                'surface_temperature_celcius': 59}],
  'max_wind_speed': 21.0,
  'precipitation': '0.00I',
  'relative_humidity': 52.9,
  'station': '948701',
  'windspeed_knots': 17.0},
]
</pre>

### 1. A python program that will read the data from hotspot_historic.csv and climate_historic.csv and load them to the new database

First, import all required libraries. 

In [1]:
import pymongo
from pymongo import MongoClient
from datetime import datetime, date
from pprint import pprint


then initial the data type for each columns in two input files.

In [2]:
#define data type for each column

integer_attr = ['air_temperature_celcius', 'surface_temperature_celcius', 'confidence', ]
float_attr = ['relative_humidity', 'windspeed_knots', 'max_wind_speed', 'latitude', 'longitude']
date_arrt = ['date']
datetime_attr = ['datetime']

The input data of the data insertion method using pymongo is a python list of dictionary. Thus, the function for transform CSV file into a list of python dictionary is needed. The following function will read the CSV data and transform each record into a python dict by using the column name on the first line as a key for the python dictionary. Then all dictionary object of record will be stored in a list.   

In [3]:
def csv2dict(file_path, delim):
    
    header_names = []
    is_header_first = True
    
    result = []
    
    for line in open(file_path):
        tmp_row = {}
        
        line = line.strip()
        
        if len(line) == 0:
            continue
            
        if is_header_first:
            header_names = line.split(delim)
            is_header_first = False
            continue
                
        tmp_data = line.split(delim)

        for i in range(len(header_names)):
            #the string data will be re-cating into the right data type using the column name and data type that declared on above  
            if header_names[i].strip() in integer_attr:
                tmp_row[header_names[i].strip()] = int(tmp_data[i].strip())
            elif header_names[i].strip() in float_attr:
                tmp_row[header_names[i].strip()] = float(tmp_data[i].strip())
            elif header_names[i].strip() in date_arrt:
                tmp_row[header_names[i].strip()] = datetime.strptime(tmp_data[i].strip(), '%d/%m/%Y')
            elif header_names[i].strip() in datetime_attr:
                tmp_row[header_names[i].strip()] = datetime.strptime(tmp_data[i].strip(), '%Y-%m-%dT%H:%M:%S')
            else :
                tmp_row[header_names[i].strip()] = tmp_data[i].strip()
                
        result.append(tmp_row)
        
    return result

#### 1.1 Reading the Hotspot Dataset

In [4]:
hotspot_historic = "./hotspot_historic.csv"
hotspot_docs = csv2dict(hotspot_historic, ',')

current_id = 0

#assign the _di for each document and group _id of document by date
group_by_date = {}

for row in hotspot_docs:
        
        #assign current_id to document
        row["_id"] = "h%d"%current_id
        
        #group ing
        date = row["date"]
        if date.strftime("%Y%m%d") not in group_by_date:
            group_by_date[date.strftime("%Y%m%d")] = []
            
        #grouping by date
        group_by_date[date.strftime("%Y%m%d")].append(row)
        
        current_id += 1
        
pprint(hotspot_docs)
#pprint(group_by_date)


[{'_id': 'h0',
  'confidence': 78,
  'date': datetime.datetime(2017, 12, 27, 0, 0),
  'datetime': datetime.datetime(2017, 12, 27, 4, 16, 51),
  'latitude': -37.966,
  'longitude': 145.051,
  'surface_temperature_celcius': 68},
 {'_id': 'h1',
  'confidence': 82,
  'date': datetime.datetime(2017, 12, 27, 0, 0),
  'datetime': datetime.datetime(2017, 12, 27, 0, 2, 15),
  'latitude': -35.541,
  'longitude': 143.311,
  'surface_temperature_celcius': 63},
 {'_id': 'h2',
  'confidence': 67,
  'date': datetime.datetime(2017, 12, 27, 0, 0),
  'datetime': datetime.datetime(2017, 12, 27, 0, 2, 15),
  'latitude': -35.554,
  'longitude': 143.307,
  'surface_temperature_celcius': 53},
 {'_id': 'h3',
  'confidence': 86,
  'date': datetime.datetime(2017, 12, 27, 0, 0),
  'datetime': datetime.datetime(2017, 12, 27, 0, 2, 14),
  'latitude': -35.543,
  'longitude': 143.316,
  'surface_temperature_celcius': 67},
 {'_id': 'h4',
  'confidence': 80,
  'date': datetime.datetime(2017, 12, 25, 0, 0),
  'datetime

  'datetime': datetime.datetime(2017, 11, 5, 4, 41, 19),
  'latitude': -37.331,
  'longitude': 143.122,
  'surface_temperature_celcius': 65},
 {'_id': 'h120',
  'confidence': 89,
  'date': datetime.datetime(2017, 11, 5, 0, 0),
  'datetime': datetime.datetime(2017, 11, 5, 4, 41, 19),
  'latitude': -37.329,
  'longitude': 143.136,
  'surface_temperature_celcius': 62},
 {'_id': 'h121',
  'confidence': 81,
  'date': datetime.datetime(2017, 10, 28, 0, 0),
  'datetime': datetime.datetime(2017, 10, 28, 3, 52, 17),
  'latitude': -37.34,
  'longitude': 143.123,
  'surface_temperature_celcius': 56},
 {'_id': 'h122',
  'confidence': 62,
  'date': datetime.datetime(2017, 10, 27, 0, 0),
  'datetime': datetime.datetime(2017, 10, 27, 4, 47, 31),
  'latitude': -36.748,
  'longitude': 144.168,
  'surface_temperature_celcius': 44},
 {'_id': 'h123',
  'confidence': 78,
  'date': datetime.datetime(2017, 10, 27, 0, 0),
  'datetime': datetime.datetime(2017, 10, 27, 4, 47, 31),
  'latitude': -36.752,
  'long

  'confidence': 68,
  'date': datetime.datetime(2017, 9, 24, 0, 0),
  'datetime': datetime.datetime(2017, 9, 24, 4, 4, 18),
  'latitude': -37.465,
  'longitude': 148.153,
  'surface_temperature_celcius': 63},
 {'_id': 'h232',
  'confidence': 64,
  'date': datetime.datetime(2017, 9, 24, 0, 0),
  'datetime': datetime.datetime(2017, 9, 24, 4, 4, 18),
  'latitude': -37.45,
  'longitude': 148.126,
  'surface_temperature_celcius': 69},
 {'_id': 'h233',
  'confidence': 86,
  'date': datetime.datetime(2017, 9, 24, 0, 0),
  'datetime': datetime.datetime(2017, 9, 24, 4, 4, 18),
  'latitude': -37.436,
  'longitude': 148.088,
  'surface_temperature_celcius': 76},
 {'_id': 'h234',
  'confidence': 73,
  'date': datetime.datetime(2017, 9, 24, 0, 0),
  'datetime': datetime.datetime(2017, 9, 24, 4, 4, 18),
  'latitude': -37.444,
  'longitude': 148.101,
  'surface_temperature_celcius': 66},
 {'_id': 'h235',
  'confidence': 61,
  'date': datetime.datetime(2017, 9, 24, 0, 0),
  'datetime': datetime.dateti

 {'_id': 'h331',
  'confidence': 53,
  'date': datetime.datetime(2017, 6, 18, 0, 0),
  'datetime': datetime.datetime(2017, 6, 18, 4, 14),
  'latitude': -37.8387,
  'longitude': 144.6301,
  'surface_temperature_celcius': 39},
 {'_id': 'h332',
  'confidence': 66,
  'date': datetime.datetime(2017, 6, 16, 0, 0),
  'datetime': datetime.datetime(2017, 6, 16, 4, 34, 30),
  'latitude': -37.3583,
  'longitude': 143.0203,
  'surface_temperature_celcius': 43},
 {'_id': 'h333',
  'confidence': 64,
  'date': datetime.datetime(2017, 6, 16, 0, 0),
  'datetime': datetime.datetime(2017, 6, 16, 4, 26, 10),
  'latitude': -36.7084,
  'longitude': 142.7354,
  'surface_temperature_celcius': 42},
 {'_id': 'h334',
  'confidence': 61,
  'date': datetime.datetime(2017, 6, 14, 0, 0),
  'datetime': datetime.datetime(2017, 6, 14, 4, 44, 10),
  'latitude': -36.2669,
  'longitude': 143.1906,
  'surface_temperature_celcius': 45},
 {'_id': 'h335',
  'confidence': 94,
  'date': datetime.datetime(2017, 6, 14, 0, 0),
  '

 {'_id': 'h431',
  'confidence': 55,
  'date': datetime.datetime(2017, 5, 18, 0, 0),
  'datetime': datetime.datetime(2017, 5, 18, 0, 43, 10),
  'latitude': -37.164,
  'longitude': 148.915,
  'surface_temperature_celcius': 42},
 {'_id': 'h432',
  'confidence': 66,
  'date': datetime.datetime(2017, 5, 18, 0, 0),
  'datetime': datetime.datetime(2017, 5, 18, 0, 38, 10),
  'latitude': -37.1521,
  'longitude': 148.9446,
  'surface_temperature_celcius': 46},
 {'_id': 'h433',
  'confidence': 75,
  'date': datetime.datetime(2017, 5, 18, 0, 0),
  'datetime': datetime.datetime(2017, 5, 18, 0, 37, 20),
  'latitude': -37.166,
  'longitude': 148.9436,
  'surface_temperature_celcius': 49},
 {'_id': 'h434',
  'confidence': 79,
  'date': datetime.datetime(2017, 5, 17, 0, 0),
  'datetime': datetime.datetime(2017, 5, 17, 4, 14, 10),
  'latitude': -36.1841,
  'longitude': 145.1836,
  'surface_temperature_celcius': 52},
 {'_id': 'h435',
  'confidence': 53,
  'date': datetime.datetime(2017, 5, 16, 0, 0),
  

...

  'longitude': 141.3614,
  'surface_temperature_celcius': 59},
 {'_id': 'h2197',
  'confidence': 90,
  'date': datetime.datetime(2017, 4, 6, 0, 0),
  'datetime': datetime.datetime(2017, 4, 6, 4, 22, 10),
  'latitude': -37.5788,
  'longitude': 148.592,
  'surface_temperature_celcius': 67},
 {'_id': 'h2198',
  'confidence': 75,
  'date': datetime.datetime(2017, 4, 6, 0, 0),
  'datetime': datetime.datetime(2017, 4, 6, 4, 21, 40),
  'latitude': -35.8065,
  'longitude': 142.1209,
  'surface_temperature_celcius': 53},
 {'_id': 'h2199',
  'confidence': 90,
  'date': datetime.datetime(2017, 4, 6, 0, 0),
  'datetime': datetime.datetime(2017, 4, 6, 4, 21, 30),
  'latitude': -36.6384,
  'longitude': 142.4075,
  'surface_temperature_celcius': 66},
 {'_id': 'h2200',
  'confidence': 74,
  'date': datetime.datetime(2017, 4, 6, 0, 0),
  'datetime': datetime.datetime(2017, 4, 6, 4, 21, 30),
  'latitude': -35.1491,
  'longitude': 141.8073,
  'surface_temperature_celcius': 54},
 {'_id': 'h2201',
  'confi

  'confidence': 81,
  'date': datetime.datetime(2017, 4, 5, 0, 0),
  'datetime': datetime.datetime(2017, 4, 5, 3, 40, 20),
  'latitude': -37.7916,
  'longitude': 148.4098,
  'surface_temperature_celcius': 54},
 {'_id': 'h2298',
  'confidence': 89,
  'date': datetime.datetime(2017, 4, 5, 0, 0),
  'datetime': datetime.datetime(2017, 4, 5, 3, 39, 30),
  'latitude': -37.4471,
  'longitude': 147.6331,
  'surface_temperature_celcius': 65},
 {'_id': 'h2299',
  'confidence': 79,
  'date': datetime.datetime(2017, 4, 5, 0, 0),
  'datetime': datetime.datetime(2017, 4, 5, 3, 39, 30),
  'latitude': -37.4024,
  'longitude': 145.5918,
  'surface_temperature_celcius': 52},
 {'_id': 'h2300',
  'confidence': 82,
  'date': datetime.datetime(2017, 4, 5, 0, 0),
  'datetime': datetime.datetime(2017, 4, 5, 3, 39, 30),
  'latitude': -37.79,
  'longitude': 143.3801,
  'surface_temperature_celcius': 62},
 {'_id': 'h2301',
  'confidence': 90,
  'date': datetime.datetime(2017, 4, 5, 0, 0),
  'datetime': datetime.

 {'_id': 'h2397',
  'confidence': 74,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  'datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
  'latitude': -35.4439,
  'longitude': 143.2005,
  'surface_temperature_celcius': 54},
 {'_id': 'h2398',
  'confidence': 67,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  'datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
  'latitude': -36.6228,
  'longitude': 142.428,
  'surface_temperature_celcius': 43},
 {'_id': 'h2399',
  'confidence': 81,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  'datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
  'latitude': -36.2205,
  'longitude': 141.7933,
  'surface_temperature_celcius': 54},
 {'_id': 'h2400',
  'confidence': 60,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  'datetime': datetime.datetime(2017, 4, 4, 4, 32, 40),
  'latitude': -38.1177,
  'longitude': 143.6409,
  'surface_temperature_celcius': 40},
 {'_id': 'h2401',
  'confidence': 100,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  '

 {'_id': 'h2497',
  'confidence': 80,
  'date': datetime.datetime(2017, 3, 31, 0, 0),
  'datetime': datetime.datetime(2017, 3, 31, 4, 58),
  'latitude': -36.1558,
  'longitude': 145.9723,
  'surface_temperature_celcius': 53},
 {'_id': 'h2498',
  'confidence': 68,
  'date': datetime.datetime(2017, 3, 31, 0, 0),
  'datetime': datetime.datetime(2017, 3, 31, 4, 57, 50),
  'latitude': -36.1649,
  'longitude': 145.9352,
  'surface_temperature_celcius': 44},
 {'_id': 'h2499',
  'confidence': 68,
  'date': datetime.datetime(2017, 3, 31, 0, 0),
  'datetime': datetime.datetime(2017, 3, 31, 4, 57, 40),
  'latitude': -36.0436,
  'longitude': 145.7964,
  'surface_temperature_celcius': 44},
 {'_id': 'h2500',
  'confidence': 70,
  'date': datetime.datetime(2017, 3, 31, 0, 0),
  'datetime': datetime.datetime(2017, 3, 31, 4, 57, 40),
  'latitude': -36.0362,
  'longitude': 145.7984,
  'surface_temperature_celcius': 45},
 {'_id': 'h2501',
  'confidence': 71,
  'date': datetime.datetime(2017, 3, 31, 0, 0)

 {'_id': 'h2597',
  'confidence': 55,
  'date': datetime.datetime(2017, 3, 24, 0, 0),
  'datetime': datetime.datetime(2017, 3, 24, 4, 56, 10),
  'latitude': -37.1684,
  'longitude': 142.8183,
  'surface_temperature_celcius': 48},
 {'_id': 'h2598',
  'confidence': 85,
  'date': datetime.datetime(2017, 3, 19, 0, 0),
  'datetime': datetime.datetime(2017, 3, 19, 4, 39, 50),
  'latitude': -37.6605,
  'longitude': 142.1116,
  'surface_temperature_celcius': 60},
 {'_id': 'h2599',
  'confidence': 58,
  'date': datetime.datetime(2017, 3, 19, 0, 0),
  'datetime': datetime.datetime(2017, 3, 19, 4, 39, 20),
  'latitude': -37.9164,
  'longitude': 143.4866,
  'surface_temperature_celcius': 54},
 {'_id': 'h2600',
  'confidence': 96,
  'date': datetime.datetime(2017, 3, 19, 0, 0),
  'datetime': datetime.datetime(2017, 3, 19, 4, 39, 20),
  'latitude': -37.6622,
  'longitude': 142.1001,
  'surface_temperature_celcius': 79},
 {'_id': 'h2601',
  'confidence': 63,
  'date': datetime.datetime(2017, 3, 19, 0

#### 1.2 Reading the Climate Dataset

In [5]:
climate_historic = "./climate_historic.csv"
climate_docs = csv2dict(climate_historic, ',')

current_id = 0

#assign the _di for each document and group _id of document by date

for row in climate_docs:
        #assign current_id to document
        row["_id"] = "c%d"%current_id
        date = row["date"]
        
        row["hotspots"] = group_by_date[date.strftime("%Y%m%d")] if date.strftime("%Y%m%d") in group_by_date else []
        current_id += 1
        
pprint(climate_docs)


[{'_id': 'c0',
  'air_temperature_celcius': 19,
  'date': datetime.datetime(2016, 12, 31, 0, 0),
  'hotspots': [],
  'max_wind_speed': 11.1,
  'precipitation': '0.00I',
  'relative_humidity': 56.8,
  'station': '948700',
  'windspeed_knots': 7.9},
 {'_id': 'c1',
  'air_temperature_celcius': 15,
  'date': datetime.datetime(2017, 1, 2, 0, 0),
  'hotspots': [],
  'max_wind_speed': 13.0,
  'precipitation': '0.02G',
  'relative_humidity': 50.7,
  'station': '948700',
  'windspeed_knots': 9.2},
 {'_id': 'c2',
  'air_temperature_celcius': 16,
  'date': datetime.datetime(2017, 1, 3, 0, 0),
  'hotspots': [],
  'max_wind_speed': 15.0,
  'precipitation': '0.00G',
  'relative_humidity': 53.6,
  'station': '948700',
  'windspeed_knots': 8.1},
 {'_id': 'c3',
  'air_temperature_celcius': 24,
  'date': datetime.datetime(2017, 1, 4, 0, 0),
  'hotspots': [],
  'max_wind_speed': 14.0,
  'precipitation': '0.00I',
  'relative_humidity': 61.6,
  'station': '948700',
  'windspeed_knots': 7.7},
 {'_id': 'c4',

                'date': datetime.datetime(2017, 3, 10, 0, 0),
                'datetime': datetime.datetime(2017, 3, 10, 4, 48, 40),
                'latitude': -37.223,
                'longitude': 147.9431,
                'surface_temperature_celcius': 105},
               {'_id': 'h2653',
                'confidence': 100,
                'date': datetime.datetime(2017, 3, 10, 0, 0),
                'datetime': datetime.datetime(2017, 3, 10, 4, 46, 20),
                'latitude': -37.2252,
                'longitude': 147.9363,
                'surface_temperature_celcius': 109},
               {'_id': 'h2654',
                'confidence': 94,
                'date': datetime.datetime(2017, 3, 10, 0, 0),
                'datetime': datetime.datetime(2017, 3, 10, 4, 45, 30),
                'latitude': -37.2284,
                'longitude': 147.9187,
                'surface_temperature_celcius': 73},
               {'_id': 'h2655',
                'confidence': 97,
              

                'date': datetime.datetime(2017, 3, 19, 0, 0),
                'datetime': datetime.datetime(2017, 3, 19, 4, 32, 50),
                'latitude': -37.9277,
                'longitude': 143.477,
                'surface_temperature_celcius': 55},
               {'_id': 'h2613',
                'confidence': 100,
                'date': datetime.datetime(2017, 3, 19, 0, 0),
                'datetime': datetime.datetime(2017, 3, 19, 4, 32, 40),
                'latitude': -37.4224,
                'longitude': 147.0511,
                'surface_temperature_celcius': 113},
               {'_id': 'h2614',
                'confidence': 92,
                'date': datetime.datetime(2017, 3, 19, 0, 0),
                'datetime': datetime.datetime(2017, 3, 19, 4, 32, 40),
                'latitude': -37.6625,
                'longitude': 144.287,
                'surface_temperature_celcius': 70},
               {'_id': 'h2615',
                'confidence': 100,
               

                'longitude': 140.9956,
                'surface_temperature_celcius': 51},
               {'_id': 'h2562',
                'confidence': 83,
                'date': datetime.datetime(2017, 3, 28, 0, 0),
                'datetime': datetime.datetime(2017, 3, 28, 4, 26, 50),
                'latitude': -35.0821,
                'longitude': 142.4532,
                'surface_temperature_celcius': 57},
               {'_id': 'h2563',
                'confidence': 78,
                'date': datetime.datetime(2017, 3, 28, 0, 0),
                'datetime': datetime.datetime(2017, 3, 28, 4, 26, 50),
                'latitude': -36.6783,
                'longitude': 141.1415,
                'surface_temperature_celcius': 51},
               {'_id': 'h2564',
                'confidence': 85,
                'date': datetime.datetime(2017, 3, 28, 0, 0),
                'datetime': datetime.datetime(2017, 3, 28, 4, 26, 50),
                'latitude': -34.3616,
                

               {'_id': 'h2458',
                'confidence': 70,
                'date': datetime.datetime(2017, 4, 3, 0, 0),
                'datetime': datetime.datetime(2017, 4, 3, 3, 50, 40),
                'latitude': -37.7583,
                'longitude': 143.41,
                'surface_temperature_celcius': 45},
               {'_id': 'h2459',
                'confidence': 69,
                'date': datetime.datetime(2017, 4, 3, 0, 0),
                'datetime': datetime.datetime(2017, 4, 3, 3, 50, 40),
                'latitude': -37.7836,
                'longitude': 142.9437,
                'surface_temperature_celcius': 44},
               {'_id': 'h2460',
                'confidence': 99,
                'date': datetime.datetime(2017, 4, 3, 0, 0),
                'datetime': datetime.datetime(2017, 4, 3, 3, 50, 40),
                'latitude': -37.7236,
                'longitude': 143.3995,
                'surface_temperature_celcius': 86},
               {'_id': '

...

                'latitude': -37.475,
                'longitude': 148.134,
                'surface_temperature_celcius': 40},
               {'_id': 'h188',
                'confidence': 83,
                'date': datetime.datetime(2017, 10, 2, 0, 0),
                'datetime': datetime.datetime(2017, 10, 2, 4, 53, 10),
                'latitude': -37.47,
                'longitude': 148.089,
                'surface_temperature_celcius': 50},
               {'_id': 'h189',
                'confidence': 71,
                'date': datetime.datetime(2017, 10, 2, 0, 0),
                'datetime': datetime.datetime(2017, 10, 2, 4, 53, 10),
                'latitude': -37.485,
                'longitude': 148.095,
                'surface_temperature_celcius': 43},
               {'_id': 'h190',
                'confidence': 85,
                'date': datetime.datetime(2017, 10, 2, 0, 0),
                'datetime': datetime.datetime(2017, 10, 2, 4, 53, 9),
                'latitude':

 {'_id': 'c297',
  'air_temperature_celcius': 14,
  'date': datetime.datetime(2017, 10, 25, 0, 0),
  'hotspots': [],
  'max_wind_speed': 12.0,
  'precipitation': '0.00G',
  'relative_humidity': 51.0,
  'station': '948702',
  'windspeed_knots': 9.1},
 {'_id': 'c298',
  'air_temperature_celcius': 13,
  'date': datetime.datetime(2017, 10, 26, 0, 0),
  'hotspots': [{'_id': 'h127',
                'confidence': 77,
                'date': datetime.datetime(2017, 10, 26, 0, 0),
                'datetime': datetime.datetime(2017, 10, 26, 13, 30, 31),
                'latitude': -35.325,
                'longitude': 143.497,
                'surface_temperature_celcius': 36},
               {'_id': 'h128',
                'confidence': 100,
                'date': datetime.datetime(2017, 10, 26, 0, 0),
                'datetime': datetime.datetime(2017, 10, 26, 13, 30, 31),
                'latitude': -35.321,
                'longitude': 143.502,
                'surface_temperature_celcius':

                'surface_temperature_celcius': 53},
               {'_id': 'h97',
                'confidence': 58,
                'date': datetime.datetime(2017, 11, 12, 0, 0),
                'datetime': datetime.datetime(2017, 11, 12, 4, 47, 16),
                'latitude': -37.864,
                'longitude': 144.174,
                'surface_temperature_celcius': 41},
               {'_id': 'h98',
                'confidence': 56,
                'date': datetime.datetime(2017, 11, 12, 0, 0),
                'datetime': datetime.datetime(2017, 11, 12, 4, 46, 53),
                'latitude': -38.912,
                'longitude': 146.272,
                'surface_temperature_celcius': 48},
               {'_id': 'h99',
                'confidence': 67,
                'date': datetime.datetime(2017, 11, 12, 0, 0),
                'datetime': datetime.datetime(2017, 11, 12, 0, 33, 15),
                'latitude': -36.942,
                'longitude': 143.292,
                'surfa

                'datetime': datetime.datetime(2017, 11, 30, 4, 34, 57),
                'latitude': -37.623,
                'longitude': 149.284,
                'surface_temperature_celcius': 61},
               {'_id': 'h60',
                'confidence': 99,
                'date': datetime.datetime(2017, 11, 30, 0, 0),
                'datetime': datetime.datetime(2017, 11, 30, 4, 34, 57),
                'latitude': -37.6,
                'longitude': 149.325,
                'surface_temperature_celcius': 86},
               {'_id': 'h61',
                'confidence': 69,
                'date': datetime.datetime(2017, 11, 30, 0, 0),
                'datetime': datetime.datetime(2017, 11, 30, 4, 34, 57),
                'latitude': -37.61,
                'longitude': 149.279,
                'surface_temperature_celcius': 48},
               {'_id': 'h62',
                'confidence': 60,
                'date': datetime.datetime(2017, 11, 30, 0, 0),
                'datetime

                'surface_temperature_celcius': 53},
               {'_id': 'h3',
                'confidence': 86,
                'date': datetime.datetime(2017, 12, 27, 0, 0),
                'datetime': datetime.datetime(2017, 12, 27, 0, 2, 14),
                'latitude': -35.543,
                'longitude': 143.316,
                'surface_temperature_celcius': 67}],
  'max_wind_speed': 15.9,
  'precipitation': '0.00I',
  'relative_humidity': 58.3,
  'station': '948702',
  'windspeed_knots': 9.3},
 {'_id': 'c361',
  'air_temperature_celcius': 21,
  'date': datetime.datetime(2017, 12, 28, 0, 0),
  'hotspots': [],
  'max_wind_speed': 11.1,
  'precipitation': '0.00I',
  'relative_humidity': 61.1,
  'station': '948702',
  'windspeed_knots': 6.6},
 {'_id': 'c362',
  'air_temperature_celcius': 19,
  'date': datetime.datetime(2017, 12, 29, 0, 0),
  'hotspots': [],
  'max_wind_speed': 14.0,
  'precipitation': '0.63G',
  'relative_humidity': 59.7,
  'station': '948702',
  'windspeed_knot

#### 1.3 Connect to MongoDB

In [6]:
#connection to mongod
client = MongoClient('localhost', 27017) 
db = client.fit5148_assignment_db


#### 1.4 Select the collection and insert the data

In [7]:
#select collection for climate historic
climate_collection = db.climate

#make sure the collection is empty by delete all docs in this collection 
climate_collection.delete_many({})

#insert climate data
climate_collection.insert_many(climate_docs)


Display the data that inserted to MongoDB

In [8]:
for tmp_climate in climate_collection.find():
    pprint(tmp_climate)


{'_id': 'c0',
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2016, 12, 31, 0, 0),
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': '0.00I',
 'relative_humidity': 56.8,
 'station': '948700',
 'windspeed_knots': 7.9}
{'_id': 'c1',
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2017, 1, 2, 0, 0),
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': '0.02G',
 'relative_humidity': 50.7,
 'station': '948700',
 'windspeed_knots': 9.2}
{'_id': 'c2',
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2017, 1, 3, 0, 0),
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': '0.00G',
 'relative_humidity': 53.6,
 'station': '948700',
 'windspeed_knots': 8.1}
{'_id': 'c3',
 'air_temperature_celcius': 24,
 'date': datetime.datetime(2017, 1, 4, 0, 0),
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': '0.00I',
 'relative_humidity': 61.6,
 'station': '948700',
 'windspeed_knots': 7.7}
{'_id': 'c4',
 'air_temperature_celcius': 24,
 'date':

               'surface_temperature_celcius': 55},
              {'_id': 'h2643',
               'confidence': 96,
               'date': datetime.datetime(2017, 3, 14, 0, 0),
               'datetime': datetime.datetime(2017, 3, 14, 4, 14, 30),
               'latitude': -34.3554,
               'longitude': 141.5413,
               'surface_temperature_celcius': 79},
              {'_id': 'h2644',
               'confidence': 94,
               'date': datetime.datetime(2017, 3, 14, 0, 0),
               'datetime': datetime.datetime(2017, 3, 14, 4, 14, 30),
               'latitude': -34.3555,
               'longitude': 141.5495,
               'surface_temperature_celcius': 75}],
 'max_wind_speed': 13.0,
 'precipitation': '0.00I',
 'relative_humidity': 58.3,
 'station': '948701',
 'windspeed_knots': 7.1}
{'_id': 'c73',
 'air_temperature_celcius': 28,
 'date': datetime.datetime(2017, 3, 15, 0, 0),
 'hotspots': [{'_id': 'h2628',
               'confidence': 76,
               'date'

               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -36.7791,
               'longitude': 141.5711,
               'surface_temperature_celcius': 46},
              {'_id': 'h2526',
               'confidence': 98,
               'date': datetime.datetime(2017, 3, 28, 0, 0),
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -34.3526,
               'longitude': 141.6458,
               'surface_temperature_celcius': 82},
              {'_id': 'h2527',
               'confidence': 81,
               'date': datetime.datetime(2017, 3, 28, 0, 0),
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -36.8189,
               'longitude': 142.73,
               'surface_temperature_celcius': 54},
              {'_id': 'h2528',
               'confidence': 97,
               'date': datetime.datetime(2017, 3, 28, 0, 0),
               'datetime': datetime.dat

               'date': datetime.datetime(2017, 4, 2, 0, 0),
               'datetime': datetime.datetime(2017, 4, 2, 4, 45, 10),
               'latitude': -36.6688,
               'longitude': 145.2635,
               'surface_temperature_celcius': 47},
              {'_id': 'h2480',
               'confidence': 63,
               'date': datetime.datetime(2017, 4, 2, 0, 0),
               'datetime': datetime.datetime(2017, 4, 2, 4, 45, 10),
               'latitude': -36.494,
               'longitude': 145.4522,
               'surface_temperature_celcius': 41},
              {'_id': 'h2481',
               'confidence': 60,
               'date': datetime.datetime(2017, 4, 2, 0, 0),
               'datetime': datetime.datetime(2017, 4, 2, 0, 24, 50),
               'latitude': -37.324,
               'longitude': 146.1647,
               'surface_temperature_celcius': 46}],
 'max_wind_speed': 15.0,
 'precipitation': '0.00I',
 'relative_humidity': 43.4,
 'station': '948701',
 'wind

               'datetime': datetime.datetime(2017, 4, 4, 4, 41),
               'latitude': -36.6936,
               'longitude': 145.2788,
               'surface_temperature_celcius': 61},
              {'_id': 'h2324',
               'confidence': 100,
               'date': datetime.datetime(2017, 4, 4, 0, 0),
               'datetime': datetime.datetime(2017, 4, 4, 4, 40),
               'latitude': -36.1057,
               'longitude': 141.7608,
               'surface_temperature_celcius': 120},
              {'_id': 'h2325',
               'confidence': 90,
               'date': datetime.datetime(2017, 4, 4, 0, 0),
               'datetime': datetime.datetime(2017, 4, 4, 4, 39, 50),
               'latitude': -35.4033,
               'longitude': 143.267,
               'surface_temperature_celcius': 67},
              {'_id': 'h2326',
               'confidence': 64,
               'date': datetime.datetime(2017, 4, 4, 0, 0),
               'datetime': datetime.datetime(2017,

...

               'longitude': 148.107,
               'surface_temperature_celcius': 50},
              {'_id': 'h249',
               'confidence': 75,
               'date': datetime.datetime(2017, 9, 23, 0, 0),
               'datetime': datetime.datetime(2017, 9, 23, 12, 47, 12),
               'latitude': -37.391,
               'longitude': 148.066,
               'surface_temperature_celcius': 39},
              {'_id': 'h250',
               'confidence': 100,
               'date': datetime.datetime(2017, 9, 23, 0, 0),
               'datetime': datetime.datetime(2017, 9, 23, 12, 47, 12),
               'latitude': -37.399,
               'longitude': 148.081,
               'surface_temperature_celcius': 84},
              {'_id': 'h251',
               'confidence': 100,
               'date': datetime.datetime(2017, 9, 23, 0, 0),
               'datetime': datetime.datetime(2017, 9, 23, 12, 47, 12),
               'latitude': -37.401,
               'longitude': 148.067,
    

              {'_id': 'h198',
               'confidence': 89,
               'date': datetime.datetime(2017, 10, 1, 0, 0),
               'datetime': datetime.datetime(2017, 10, 1, 4, 10, 28),
               'latitude': -37.467,
               'longitude': 148.127,
               'surface_temperature_celcius': 61},
              {'_id': 'h199',
               'confidence': 64,
               'date': datetime.datetime(2017, 10, 1, 0, 0),
               'datetime': datetime.datetime(2017, 10, 1, 4, 10, 28),
               'latitude': -37.46,
               'longitude': 148.113,
               'surface_temperature_celcius': 52},
              {'_id': 'h200',
               'confidence': 78,
               'date': datetime.datetime(2017, 10, 1, 0, 0),
               'datetime': datetime.datetime(2017, 10, 1, 4, 10, 28),
               'latitude': -37.453,
               'longitude': 148.099,
               'surface_temperature_celcius': 44}],
 'max_wind_speed': 11.1,
 'precipitation': '0.

              {'_id': 'h147',
               'confidence': 73,
               'date': datetime.datetime(2017, 10, 17, 0, 0),
               'datetime': datetime.datetime(2017, 10, 17, 4, 10, 59),
               'latitude': -35.937,
               'longitude': 145.607,
               'surface_temperature_celcius': 51},
              {'_id': 'h148',
               'confidence': 91,
               'date': datetime.datetime(2017, 10, 17, 0, 0),
               'datetime': datetime.datetime(2017, 10, 17, 4, 10, 45),
               'latitude': -37.247,
               'longitude': 141.278,
               'surface_temperature_celcius': 68},
              {'_id': 'h149',
               'confidence': 58,
               'date': datetime.datetime(2017, 10, 17, 0, 0),
               'datetime': datetime.datetime(2017, 10, 17, 4, 10, 38),
               'latitude': -37.587,
               'longitude': 142.479,
               'surface_temperature_celcius': 41},
              {'_id': 'h150',
          

 'air_temperature_celcius': 18,
 'date': datetime.datetime(2017, 11, 16, 0, 0),
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': '0.28G',
 'relative_humidity': 61.5,
 'station': '948702',
 'windspeed_knots': 5.5}
{'_id': 'c320',
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2017, 11, 17, 0, 0),
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': '0.28G',
 'relative_humidity': 58.7,
 'station': '948702',
 'windspeed_knots': 5.5}
{'_id': 'c321',
 'air_temperature_celcius': 20,
 'date': datetime.datetime(2017, 11, 18, 0, 0),
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': '0.01G',
 'relative_humidity': 58.4,
 'station': '948702',
 'windspeed_knots': 4.7}
{'_id': 'c322',
 'air_temperature_celcius': 22,
 'date': datetime.datetime(2017, 11, 19, 0, 0),
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': '0.00G',
 'relative_humidity': 57.0,
 'station': '948702',
 'windspeed_knots': 8.5}
{'_id': 'c323',
 'air_temperature_celcius': 24,
 'date':

               'datetime': datetime.datetime(2017, 12, 15, 13, 17, 17),
               'latitude': -37.627,
               'longitude': 149.33,
               'surface_temperature_celcius': 42},
              {'_id': 'h23',
               'confidence': 78,
               'date': datetime.datetime(2017, 12, 15, 0, 0),
               'datetime': datetime.datetime(2017, 12, 15, 13, 17, 17),
               'latitude': -37.658,
               'longitude': 149.339,
               'surface_temperature_celcius': 36},
              {'_id': 'h24',
               'confidence': 51,
               'date': datetime.datetime(2017, 12, 15, 0, 0),
               'datetime': datetime.datetime(2017, 12, 15, 13, 17, 17),
               'latitude': -37.623,
               'longitude': 149.323,
               'surface_temperature_celcius': 38},
              {'_id': 'h25',
               'confidence': 65,
               'date': datetime.datetime(2017, 12, 15, 0, 0),
               'datetime': datetime.datet

#### a. Find climate data on 10th December 2017.

In [9]:
pipeline = []

results = climate_collection.find({"date": datetime(2017, 12, 10)}, {"_id":0, "hotspots":0})
for doc in results:
    #just for pretty print on the date
    doc["date"] = str(doc["date"])
    pprint(doc)

{'air_temperature_celcius': 17,
 'date': '2017-12-10 00:00:00',
 'max_wind_speed': 14.0,
 'precipitation': '0.00I',
 'relative_humidity': 53.5,
 'station': '948702',
 'windspeed_knots': 7.3}


#### b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [10]:
#results = climate_collection.find({"surface_temperature_celcius":{"$lte":100, "$gte":65}}, {"_id":0, "date":0, "datetime":0} )

pipeline = [{"$unwind": "$hotspots"},
            {"$match": { "$and": [{"hotspots.surface_temperature_celcius":{"$lte":100}}, {"hotspots.surface_temperature_celcius":{"$gte":65}}]}}, 
            {"$project": {
                "_id":0,  
                "latitude": "$hotspots.latitude", 
                "longitude": "$hotspots.longitude", 
                "surface_temperature_celcius": "$hotspots.surface_temperature_celcius", 
                "confidence": "$hotspots.confidence" 
            }}
           ]

results = climate_collection.aggregate(pipeline)

for doc in results:
    pprint(doc)    


{'confidence': 94,
 'latitude': -37.2284,
 'longitude': 147.9187,
 'surface_temperature_celcius': 73}
{'confidence': 97,
 'latitude': -37.6572,
 'longitude': 142.0703,
 'surface_temperature_celcius': 80}
{'confidence': 84,
 'latitude': -37.0193,
 'longitude': 148.1459,
 'surface_temperature_celcius': 71}
{'confidence': 100,
 'latitude': -37.4229,
 'longitude': 147.027,
 'surface_temperature_celcius': 99}
{'confidence': 80,
 'latitude': -37.0055,
 'longitude': 148.1582,
 'surface_temperature_celcius': 68}
{'confidence': 85,
 'latitude': -37.4128,
 'longitude': 147.0242,
 'surface_temperature_celcius': 98}
{'confidence': 90,
 'latitude': -34.357,
 'longitude': 141.5361,
 'surface_temperature_celcius': 67}
{'confidence': 93,
 'latitude': -34.3539,
 'longitude': 141.5547,
 'surface_temperature_celcius': 72}
{'confidence': 90,
 'latitude': -36.9939,
 'longitude': 148.2244,
 'surface_temperature_celcius': 68}
{'confidence': 95,
 'latitude': -36.9959,
 'longitude': 148.2118,
 'surface_tempera

{'confidence': 93,
 'latitude': -36.4489,
 'longitude': 144.7685,
 'surface_temperature_celcius': 73}
{'confidence': 96,
 'latitude': -36.9498,
 'longitude': 142.2012,
 'surface_temperature_celcius': 78}
{'confidence': 93,
 'latitude': -36.1941,
 'longitude': 143.1604,
 'surface_temperature_celcius': 72}
{'confidence': 100,
 'latitude': -36.4142,
 'longitude': 143.1077,
 'surface_temperature_celcius': 97}
{'confidence': 92,
 'latitude': -36.1924,
 'longitude': 143.1716,
 'surface_temperature_celcius': 70}
{'confidence': 89,
 'latitude': -36.0295,
 'longitude': 143.6409,
 'surface_temperature_celcius': 65}
{'confidence': 100,
 'latitude': -37.0761,
 'longitude': 141.0574,
 'surface_temperature_celcius': 89}
{'confidence': 93,
 'latitude': -35.0616,
 'longitude': 141.4417,
 'surface_temperature_celcius': 73}
{'confidence': 100,
 'latitude': -36.3999,
 'longitude': 144.737,
 'surface_temperature_celcius': 88}
{'confidence': 91,
 'latitude': -36.1748,
 'longitude': 145.7582,
 'surface_temp

 'longitude': 146.0601,
 'surface_temperature_celcius': 77}
{'confidence': 100,
 'latitude': -36.5839,
 'longitude': 143.0588,
 'surface_temperature_celcius': 99}
{'confidence': 100,
 'latitude': -36.5854,
 'longitude': 143.0462,
 'surface_temperature_celcius': 94}
{'confidence': 95,
 'latitude': -36.3014,
 'longitude': 143.203,
 'surface_temperature_celcius': 76}
{'confidence': 100,
 'latitude': -38.1917,
 'longitude': 147.0745,
 'surface_temperature_celcius': 94}
{'confidence': 95,
 'latitude': -38.1987,
 'longitude': 147.0666,
 'surface_temperature_celcius': 76}
{'confidence': 100,
 'latitude': -36.5454,
 'longitude': 144.7402,
 'surface_temperature_celcius': 96}
{'confidence': 92,
 'latitude': -36.7593,
 'longitude': 142.895,
 'surface_temperature_celcius': 69}
{'confidence': 90,
 'latitude': -36.3435,
 'longitude': 141.5513,
 'surface_temperature_celcius': 67}
{'confidence': 93,
 'latitude': -36.6942,
 'longitude': 143.8021,
 'surface_temperature_celcius': 72}
{'confidence': 100,


 'longitude': 144.1321,
 'surface_temperature_celcius': 69}
{'confidence': 91,
 'latitude': -35.6819,
 'longitude': 143.5275,
 'surface_temperature_celcius': 69}
{'confidence': 90,
 'latitude': -35.6871,
 'longitude': 143.5235,
 'surface_temperature_celcius': 68}
{'confidence': 100,
 'latitude': -36.1506,
 'longitude': 143.5671,
 'surface_temperature_celcius': 95}
{'confidence': 93,
 'latitude': -37.4984,
 'longitude': 142.9739,
 'surface_temperature_celcius': 73}
{'confidence': 96,
 'latitude': -37.253,
 'longitude': 144.3681,
 'surface_temperature_celcius': 78}
{'confidence': 92,
 'latitude': -36.9396,
 'longitude': 143.1462,
 'surface_temperature_celcius': 70}
{'confidence': 96,
 'latitude': -36.3387,
 'longitude': 141.7542,
 'surface_temperature_celcius': 77}
{'confidence': 100,
 'latitude': -36.3433,
 'longitude': 143.4369,
 'surface_temperature_celcius': 94}
{'confidence': 100,
 'latitude': -36.345,
 'longitude': 143.4256,
 'surface_temperature_celcius': 87}
{'confidence': 90,
 '

#### c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2017.


In [11]:
#option 2: using aggregate and lookup
pipeline = [{"$match": { "$or": [{"date":datetime(2017, 12, 15)}, {"date":datetime(2017, 12, 16)}]}}, 
            {"$unwind": "$hotspots"},
            {"$project": {
                "_id":0, 
                "date":1, 
                "surface_temperature_celcius": "$hotspots.surface_temperature_celcius", 
                "air_temperature_celcius":1,
                "relative_humidity":1,
                "max_wind_speed":1
            }}
           ]

results = climate_collection.aggregate(pipeline)

count = 0
for doc in results:
    #just for pretty print on the date
    doc["date"] = str(doc["date"])
    pprint(doc)
    count += 1
    

print("total result:", count, "records")

{'air_temperature_celcius': 18,
 'date': '2017-12-15 00:00:00',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 42}
{'air_temperature_celcius': 18,
 'date': '2017-12-15 00:00:00',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 36}
{'air_temperature_celcius': 18,
 'date': '2017-12-15 00:00:00',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 38}
{'air_temperature_celcius': 18,
 'date': '2017-12-15 00:00:00',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': 40}
{'air_temperature_celcius': 18,
 'date': '2017-12-16 00:00:00',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celcius': 43}
{'air_temperature_celcius': 18,
 'date': '2017-12-16 00:00:00',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celcius': 33}
{'air_temperature_celcius': 18,
 'date': '2017-12-16 00:00:00',
 'max_wind_speed':

#### d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.


In [12]:
pipeline = [{"$unwind": "$hotspots"},
            {"$match": {"hotspots.confidence": {"$lte":100, "$gte":80}}}, 
            {"$project": {
                "_id":0, 
                "datetime":"$hotspots.datetime",
                "air_temperature_celcius":1,
                "surface_temperature_celcius":"$hotspots.surface_temperature_celcius", 
                "confidence":"$hotspots.confidence"
            }}
           ]

results = climate_collection.aggregate(pipeline)

for doc in results:
    #just for pretty print on the date
    doc["datetime"] = str(doc["datetime"])
    pprint(doc)

{'air_temperature_celcius': 20,
 'confidence': 87,
 'datetime': '2017-03-06 05:06:30',
 'surface_temperature_celcius': 62}
{'air_temperature_celcius': 20,
 'confidence': 85,
 'datetime': '2017-03-06 05:06:20',
 'surface_temperature_celcius': 59}
{'air_temperature_celcius': 19,
 'confidence': 88,
 'datetime': '2017-03-07 04:16:10',
 'surface_temperature_celcius': 64}
{'air_temperature_celcius': 23,
 'confidence': 86,
 'datetime': '2017-03-09 13:23:40',
 'surface_temperature_celcius': 41}
{'air_temperature_celcius': 19,
 'confidence': 100,
 'datetime': '2017-03-10 04:48:40',
 'surface_temperature_celcius': 105}
{'air_temperature_celcius': 19,
 'confidence': 100,
 'datetime': '2017-03-10 04:46:20',
 'surface_temperature_celcius': 109}
{'air_temperature_celcius': 19,
 'confidence': 94,
 'datetime': '2017-03-10 04:45:30',
 'surface_temperature_celcius': 73}
{'air_temperature_celcius': 19,
 'confidence': 97,
 'datetime': '2017-03-10 04:45:30',
 'surface_temperature_celcius': 80}
{'air_temper

 'confidence': 83,
 'datetime': '2017-03-28 04:26:50',
 'surface_temperature_celcius': 57}
{'air_temperature_celcius': 18,
 'confidence': 85,
 'datetime': '2017-03-28 04:26:50',
 'surface_temperature_celcius': 60}
{'air_temperature_celcius': 14,
 'confidence': 82,
 'datetime': '2017-03-31 05:07:10',
 'surface_temperature_celcius': 55}
{'air_temperature_celcius': 14,
 'confidence': 87,
 'datetime': '2017-03-31 05:01:20',
 'surface_temperature_celcius': 62}
{'air_temperature_celcius': 14,
 'confidence': 82,
 'datetime': '2017-03-31 04:58:30',
 'surface_temperature_celcius': 56}
{'air_temperature_celcius': 14,
 'confidence': 80,
 'datetime': '2017-03-31 04:58:00',
 'surface_temperature_celcius': 53}
{'air_temperature_celcius': 14,
 'confidence': 86,
 'datetime': '2017-03-31 04:57:10',
 'surface_temperature_celcius': 61}
{'air_temperature_celcius': 14,
 'confidence': 81,
 'datetime': '2017-03-31 00:36:30',
 'surface_temperature_celcius': 54}
{'air_temperature_celcius': 15,
 'confidence': 9

{'air_temperature_celcius': 17,
 'confidence': 90,
 'datetime': '2017-04-05 03:39:20',
 'surface_temperature_celcius': 66}
{'air_temperature_celcius': 17,
 'confidence': 83,
 'datetime': '2017-04-05 03:39:20',
 'surface_temperature_celcius': 56}
{'air_temperature_celcius': 17,
 'confidence': 89,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 65}
{'air_temperature_celcius': 17,
 'confidence': 83,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 64}
{'air_temperature_celcius': 17,
 'confidence': 95,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 76}
{'air_temperature_celcius': 17,
 'confidence': 100,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 92}
{'air_temperature_celcius': 17,
 'confidence': 88,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 63}
{'air_temperature_celcius': 17,
 'confidence': 89,
 'datetime': '2017-04-05 03:39:10',
 'surface_temperature_celcius': 64}
{'air_temperatu

{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-04-12 05:28:10',
 'surface_temperature_celcius': 106}
{'air_temperature_celcius': 14,
 'confidence': 84,
 'datetime': '2017-04-12 05:28:00',
 'surface_temperature_celcius': 58}
{'air_temperature_celcius': 14,
 'confidence': 94,
 'datetime': '2017-04-12 05:21:40',
 'surface_temperature_celcius': 75}
{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-04-12 05:21:40',
 'surface_temperature_celcius': 98}
{'air_temperature_celcius': 14,
 'confidence': 95,
 'datetime': '2017-04-12 03:54:10',
 'surface_temperature_celcius': 77}
{'air_temperature_celcius': 14,
 'confidence': 87,
 'datetime': '2017-04-12 03:50:00',
 'surface_temperature_celcius': 63}
{'air_temperature_celcius': 14,
 'confidence': 95,
 'datetime': '2017-04-12 03:50:00',
 'surface_temperature_celcius': 75}
{'air_temperature_celcius': 14,
 'confidence': 86,
 'datetime': '2017-04-12 03:50:00',
 'surface_temperature_celcius': 61}
{'air_tempera

 'surface_temperature_celcius': 66}
{'air_temperature_celcius': 16,
 'confidence': 93,
 'datetime': '2017-04-13 04:28:10',
 'surface_temperature_celcius': 72}
{'air_temperature_celcius': 16,
 'confidence': 84,
 'datetime': '2017-04-13 04:28:10',
 'surface_temperature_celcius': 58}
{'air_temperature_celcius': 16,
 'confidence': 98,
 'datetime': '2017-04-13 04:27:40',
 'surface_temperature_celcius': 82}
{'air_temperature_celcius': 16,
 'confidence': 90,
 'datetime': '2017-04-13 04:27:40',
 'surface_temperature_celcius': 66}
{'air_temperature_celcius': 16,
 'confidence': 92,
 'datetime': '2017-04-13 04:27:30',
 'surface_temperature_celcius': 70}
{'air_temperature_celcius': 16,
 'confidence': 100,
 'datetime': '2017-04-13 04:27:30',
 'surface_temperature_celcius': 88}
{'air_temperature_celcius': 16,
 'confidence': 100,
 'datetime': '2017-04-13 04:27:20',
 'surface_temperature_celcius': 113}
{'air_temperature_celcius': 16,
 'confidence': 80,
 'datetime': '2017-04-13 04:27:20',
 'surface_tem

 'confidence': 100,
 'datetime': '2017-04-15 04:14:20',
 'surface_temperature_celcius': 100}
{'air_temperature_celcius': 15,
 'confidence': 83,
 'datetime': '2017-04-16 05:07:40',
 'surface_temperature_celcius': 57}
{'air_temperature_celcius': 15,
 'confidence': 87,
 'datetime': '2017-04-16 04:59:30',
 'surface_temperature_celcius': 61}
{'air_temperature_celcius': 15,
 'confidence': 80,
 'datetime': '2017-04-16 04:57:20',
 'surface_temperature_celcius': 53}
{'air_temperature_celcius': 15,
 'confidence': 83,
 'datetime': '2017-04-16 04:56:50',
 'surface_temperature_celcius': 57}
{'air_temperature_celcius': 15,
 'confidence': 91,
 'datetime': '2017-04-16 04:56:50',
 'surface_temperature_celcius': 68}
{'air_temperature_celcius': 15,
 'confidence': 86,
 'datetime': '2017-04-16 04:56:50',
 'surface_temperature_celcius': 61}
{'air_temperature_celcius': 13,
 'confidence': 81,
 'datetime': '2017-04-17 04:08:20',
 'surface_temperature_celcius': 55}
{'air_temperature_celcius': 13,
 'confidence':

 'surface_temperature_celcius': 58}
{'air_temperature_celcius': 20,
 'confidence': 93,
 'datetime': '2017-04-20 04:34:10',
 'surface_temperature_celcius': 72}
{'air_temperature_celcius': 20,
 'confidence': 88,
 'datetime': '2017-04-20 04:33:20',
 'surface_temperature_celcius': 63}
{'air_temperature_celcius': 20,
 'confidence': 90,
 'datetime': '2017-04-20 04:32:50',
 'surface_temperature_celcius': 67}
{'air_temperature_celcius': 20,
 'confidence': 85,
 'datetime': '2017-04-20 04:32:30',
 'surface_temperature_celcius': 60}
{'air_temperature_celcius': 20,
 'confidence': 80,
 'datetime': '2017-04-20 04:32:30',
 'surface_temperature_celcius': 53}
{'air_temperature_celcius': 20,
 'confidence': 98,
 'datetime': '2017-04-20 04:32:30',
 'surface_temperature_celcius': 84}
{'air_temperature_celcius': 20,
 'confidence': 87,
 'datetime': '2017-04-20 04:32:30',
 'surface_temperature_celcius': 62}
{'air_temperature_celcius': 20,
 'confidence': 88,
 'datetime': '2017-04-20 00:18:40',
 'surface_temper

 'confidence': 95,
 'datetime': '2017-05-04 04:48:20',
 'surface_temperature_celcius': 76}
{'air_temperature_celcius': 10,
 'confidence': 80,
 'datetime': '2017-05-04 04:48:10',
 'surface_temperature_celcius': 53}
{'air_temperature_celcius': 10,
 'confidence': 87,
 'datetime': '2017-05-04 04:48:10',
 'surface_temperature_celcius': 62}
{'air_temperature_celcius': 10,
 'confidence': 100,
 'datetime': '2017-05-04 04:48:10',
 'surface_temperature_celcius': 89}
{'air_temperature_celcius': 10,
 'confidence': 87,
 'datetime': '2017-05-04 04:46:50',
 'surface_temperature_celcius': 63}
{'air_temperature_celcius': 10,
 'confidence': 81,
 'datetime': '2017-05-04 04:46:30',
 'surface_temperature_celcius': 54}
{'air_temperature_celcius': 10,
 'confidence': 91,
 'datetime': '2017-05-04 04:46:20',
 'surface_temperature_celcius': 68}
{'air_temperature_celcius': 10,
 'confidence': 84,
 'datetime': '2017-05-04 04:46:20',
 'surface_temperature_celcius': 58}
{'air_temperature_celcius': 10,
 'confidence': 

 'surface_temperature_celcius': 60}
{'air_temperature_celcius': 9,
 'confidence': 85,
 'datetime': '2017-05-13 04:38:40',
 'surface_temperature_celcius': 60}
{'air_temperature_celcius': 10,
 'confidence': 93,
 'datetime': '2017-05-15 04:34:30',
 'surface_temperature_celcius': 73}
{'air_temperature_celcius': 10,
 'confidence': 96,
 'datetime': '2017-05-15 04:34:30',
 'surface_temperature_celcius': 78}
{'air_temperature_celcius': 10,
 'confidence': 83,
 'datetime': '2017-05-15 04:33:00',
 'surface_temperature_celcius': 57}
{'air_temperature_celcius': 10,
 'confidence': 92,
 'datetime': '2017-05-15 04:33:00',
 'surface_temperature_celcius': 70}
{'air_temperature_celcius': 10,
 'confidence': 100,
 'datetime': '2017-05-15 04:32:50',
 'surface_temperature_celcius': 101}
{'air_temperature_celcius': 10,
 'confidence': 84,
 'datetime': '2017-05-15 04:32:50',
 'surface_temperature_celcius': 58}
{'air_temperature_celcius': 10,
 'confidence': 96,
 'datetime': '2017-05-15 04:31:20',
 'surface_tempe

 'surface_temperature_celcius': 61}
{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-09-24 13:30:10',
 'surface_temperature_celcius': 47}
{'air_temperature_celcius': 14,
 'confidence': 82,
 'datetime': '2017-09-24 13:30:09',
 'surface_temperature_celcius': 38}
{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-09-24 13:30:09',
 'surface_temperature_celcius': 48}
{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-09-24 13:30:09',
 'surface_temperature_celcius': 48}
{'air_temperature_celcius': 14,
 'confidence': 84,
 'datetime': '2017-09-24 13:30:07',
 'surface_temperature_celcius': 50}
{'air_temperature_celcius': 14,
 'confidence': 100,
 'datetime': '2017-09-24 04:04:19',
 'surface_temperature_celcius': 88}
{'air_temperature_celcius': 14,
 'confidence': 86,
 'datetime': '2017-09-24 04:04:18',
 'surface_temperature_celcius': 76}
{'air_temperature_celcius': 14,
 'confidence': 84,
 'datetime': '2017-09-24 04:04:04',
 'surface_te

#### e. Find the top 10 records with the highest surface temperature (°C).


In [13]:
#Option 1 : Top 10 surface temperature
pipeline = [{"$unwind": "$hotspots"},
            {"$sort": {"hotspots.surface_temperature_celcius": -1}}, 
            {"$limit": 10},
            {"$project": {
                "_id": 0,
                "surface_temperature_celcius": "$hotspots.surface_temperature_celcius",
                "latitude": "$hotspots.latitude", 
                "longitude": "$hotspots.longitude", 
                "confidence": "$hotspots.confidence",
                "datetime":"$hotspots.datetime"
            }}
           ]

results = climate_collection.aggregate(pipeline)

for doc in results:
    #just for pretty print on the date
    doc["datetime"] = str(doc["datetime"])
    pprint(doc) 

{'confidence': 100,
 'datetime': '2017-04-18 04:52:00',
 'latitude': -38.1665,
 'longitude': 143.062,
 'surface_temperature_celcius': 124}
{'confidence': 100,
 'datetime': '2017-04-04 04:32:50',
 'latitude': -36.343,
 'longitude': 142.1986,
 'surface_temperature_celcius': 123}
{'confidence': 100,
 'datetime': '2017-05-01 04:14:20',
 'latitude': -36.9318,
 'longitude': 143.0907,
 'surface_temperature_celcius': 122}
{'confidence': 100,
 'datetime': '2017-03-18 03:50:50',
 'latitude': -37.017,
 'longitude': 148.1297,
 'surface_temperature_celcius': 121}
{'confidence': 100,
 'datetime': '2017-04-04 04:32:40',
 'latitude': -36.3521,
 'longitude': 142.2008,
 'surface_temperature_celcius': 120}
{'confidence': 100,
 'datetime': '2017-04-13 04:28:10',
 'latitude': -37.0745,
 'longitude': 141.0692,
 'surface_temperature_celcius': 120}
{'confidence': 100,
 'datetime': '2017-05-13 04:40:20',
 'latitude': -34.9938,
 'longitude': 141.876,
 'surface_temperature_celcius': 120}
{'confidence': 100,
 'da

In [14]:
#option 2, Distinct
pipeline = [{"$unwind": "$hotspots"},
            {"$group":{
                "_id": "$hotspots.surface_temperature_celcius",
                "count": { "$sum": 1 }
            }},
            
            {"$sort": {"_id": -1}}, 
            {"$limit": 10},
            {"$project": {
                "surface_temperature_celcius": "$_id",
                "count_duplicated": "$count",
                "_id": 0
            }}
           ]

results = climate_collection.aggregate(pipeline)

for doc in results:
    pprint(doc)

{'count_duplicated': 1, 'surface_temperature_celcius': 124}
{'count_duplicated': 1, 'surface_temperature_celcius': 123}
{'count_duplicated': 1, 'surface_temperature_celcius': 122}
{'count_duplicated': 1, 'surface_temperature_celcius': 121}
{'count_duplicated': 5, 'surface_temperature_celcius': 120}
{'count_duplicated': 1, 'surface_temperature_celcius': 119}
{'count_duplicated': 1, 'surface_temperature_celcius': 118}
{'count_duplicated': 2, 'surface_temperature_celcius': 117}
{'count_duplicated': 1, 'surface_temperature_celcius': 116}
{'count_duplicated': 5, 'surface_temperature_celcius': 115}


#### f. Find the number of fire in each day. You are required to only display the total number of fire and the date in the output.


In [15]:
#results = hotspot_collection.aggregate([ {'$project': {'date': 1, 'count': {'$size': '$hotspot'}}}])

pipeline = [{"$project": {"_id":0, 
                          "date":1, 
                          "number_of_fire":{"$size": "$hotspots"}
                         }
            },
            ]

results = climate_collection.aggregate(pipeline)

for doc in results:
    #just for pretty print on the date
    doc["date"] = str(doc["date"])
    pprint(doc)

{'date': '2016-12-31 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-02 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-03 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-04 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-05 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-06 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-07 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-08 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-09 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-10 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-11 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-12 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-13 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-14 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-15 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-16 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-17 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-18 00:00:00', 'number_of_fire': 0}
{'date': '2017-01-19 00:00:00', 'number_of_fir

#### g. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

Assumtion: we can calculate average surface temperature only the day that has fire, thus we can finish this task by only using the hotspot historical data


In [16]:
pipeline = [{"$project": {"_id":0, 
                          "date":1, 
                          "average_surface_temperature": {"$ifNull": [{ "$avg": "$hotspots.surface_temperature_celcius"}, 0]}
                         }
            },
            ]

results = climate_collection.aggregate(pipeline)
for doc in results:
    #just for pretty print on the date
    doc["date"] = str(doc["date"])
    pprint(doc)
    

{'average_surface_temperature': 0, 'date': '2016-12-31 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-02 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-03 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-04 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-05 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-06 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-07 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-08 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-09 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-10 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-11 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-12 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-13 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-14 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-01-15 00:00:00'}
{'average_

 'date': '2017-05-24 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-25 00:00:00'}
{'average_surface_temperature': 49.5, 'date': '2017-05-26 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-27 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-28 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-29 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-30 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-05-31 00:00:00'}
{'average_surface_temperature': 54.0, 'date': '2017-06-01 00:00:00'}
{'average_surface_temperature': 47.72727272727273,
 'date': '2017-06-02 00:00:00'}
{'average_surface_temperature': 47.0, 'date': '2017-06-03 00:00:00'}
{'average_surface_temperature': 52.22222222222222,
 'date': '2017-06-04 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-06-05 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-06-06 00:00:00'}
{'average_surface_temperature': 51.857142857142854,
 'date': '2017-

{'average_surface_temperature': 0, 'date': '2017-11-25 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-11-26 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-11-27 00:00:00'}
{'average_surface_temperature': 42.0, 'date': '2017-11-28 00:00:00'}
{'average_surface_temperature': 60.625, 'date': '2017-11-29 00:00:00'}
{'average_surface_temperature': 52.41935483870968,
 'date': '2017-11-30 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-01 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-02 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-03 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-04 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-05 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-06 00:00:00'}
{'average_surface_temperature': 0, 'date': '2017-12-07 00:00:00'}
{'average_surface_temperature': 50.6, 'date': '2017-12-08 00:00:00'}
{'average_surface_temperature': 58.25, 'date': '